<a href="https://colab.research.google.com/github/Eduardo-Mati/Projetos-integrador---Desafio-unifacisa---Modelos-de-Machine-Learning/blob/main/Q8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalação Básica
!pip install numpy==1.26.0 pandas

# Filtragem Colaborativa
!pip install scikit-surprise

# Aprendizado Profundo (Autoencoders)
!pip install tensorflow keras
# Opcional, mas útil:
!pip install tensorflow-recommenders
import pandas as pd
import numpy as np

from surprise.model_selection import train_test_split as surprise_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.5 MB/s eta 0:00:00


In [2]:
# Criando dataframe
df = pd.read_csv('avaliacoes_filmes.csv')

In [3]:
df.head(10)

,user_id,movie_id,rating,timestamp
0,1,101,5,2025-09-01
1,1,102,3,2025-09-02
2,1,103,4,2025-09-03
3,2,101,4,2025-09-01
4,2,102,2,2025-09-02
5,2,104,5,2025-09-03
6,3,101,2,2025-09-01
7,3,103,5,2025-09-02
8,3,104,3,2025-09-03
9,4,102,4,2025-09-01


In [4]:
# --- Modelo de Filtragem Colaborativa Baseado em Usuário (User-Based) ---
print("\n--- Avaliando Modelo User-Based ---")

# Um Reader é necessário para parsear o arquivo ou dataframe
# O formato do dataframe deve ser user item rating
from surprise import Reader, Dataset, accuracy
from surprise.model_selection import train_test_split as surprise_split
from surprise.prediction_algorithms import KNNBasic

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

# Divide o dataset em treino e teste usando a função do surprise
trainset, testset = surprise_split(data, test_size=.25, random_state=42)


# Configura o modelo para usar a similaridade de cossenos entre usuários
sim_options_user = {'name': 'cosine',
                    'user_based': True  # Importante: define como user-based
                   }
model_user = KNNBasic(sim_options=sim_options_user)

# Treina o modelo
model_user.fit(trainset)

# Faz as previsões no conjunto de teste
predictions_user = model_user.test(testset)

# Calcula e exibe o RMSE e MAE
print("Resultados para User-Based:")
accuracy.rmse(predictions_user)
accuracy.mae(predictions_user)


# --- Modelo de Filtragem Colaborativa Baseado em Item (Item-Based) ---
print("\n--- Avaliando Modelo Item-Based ---")

# Configura o modelo para usar a similaridade de cossenos entre itens
sim_options_item = {'name': 'cosine',
                    'user_based': False  # Importante: define como item-based
                   }
model_item = KNNBasic(sim_options=sim_options_item)

# Treina o modelo
model_item.fit(trainset)

# Faz as previsões no conjunto de teste
predictions_item = model_item.test(testset)

# Faz as previsões no conjunto de teste
predictions_item = model_item.test(testset)

# Calcula e exibe o RMSE e MAE
print("Resultados para Item-Based:")
accuracy.rmse(predictions_item)
accuracy.mae(predictions_item)


--- Avaliando Modelo User-Based ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
Resultados para User-Based:
RMSE: 1.7159
MAE:  1.3333

--- Avaliando Modelo Item-Based ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
Resultados para Item-Based:
RMSE: 1.8875
MAE:  1.6250


1.625

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Assumindo que seu DataFrame original se chama 'df_ratings'
# com colunas ['user_id', 'movie_id', 'rating']

# PRIMEIRO, VAMOS DIVIDIR OS DADOS EM TREINO E TESTE ANTES DE TUDO
# Isso garante que o teste seja feito em dados que o modelo NUNCA viu.
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# AGORA, CRIAMOS A MATRIZ USUÁRIO-ITEM APENAS COM OS DADOS DE TREINO
print("Criando a matriz usuário-item de treino...")
user_item_matrix_train = train_data.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# Para a avaliação, vamos precisar dos dados de teste no mesmo formato de matriz
user_item_matrix_test = test_data.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)

# Normalizar os dados para a rede neural aprender melhor (opcional, mas recomendado)
# A rede funciona melhor com valores pequenos, entre 0 e 1.
scaler = MinMaxScaler()
user_item_matrix_train_scaled = scaler.fit_transform(user_item_matrix_train.values)

print("Dimensões da matriz de treino:", user_item_matrix_train_scaled.shape)

Criando a matriz usuário-item de treino...
Dimensões da matriz de treino: (5, 4)


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam


# A dimensão de entrada é o número de filmes
n_movies = user_item_matrix_train_scaled.shape[1]

# --- Arquitetura da Rede ---

# Camada de Entrada: tem o mesmo número de neurônios que o número de filmes
input_layer = Input(shape=(n_movies,))

# Encoder: comprime a informação
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)

# Camada Latente (o "gargalo"): a representação mais comprimida do gosto do usuário
latent_view = Dense(32, activation='relu')(encoded)

# Decoder: tenta reconstruir a informação original a partir da camada latente
decoded = Dense(64, activation='relu')(latent_view)
decoded = Dense(128, activation='relu')(decoded)

# Camada de Saída: tem o mesmo número de neurônios da entrada, para gerar as previsões
output_layer = Dense(n_movies, activation='sigmoid')(decoded) # Sigmoid para garantir saída entre 0 e 1

# Compilando o modelo
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Mostra um resumo da arquitetura
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,924 (85.64 KB)

 Trainable params: 21,924 (85.64 KB)

 Non-trainable params: 0 (0.00 B)